In [3]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

## Loading samples

In [15]:
data = pd.read_csv("CUP_IT_train_data.csv")

In [38]:
# data.head(n=12)
print("cu_education_level -", len(data['cu_education_level'].unique()))
print("cu_empl_area -", len(data['cu_empl_area'].unique()))
print("cu_empl_level -", len(data['cu_empl_level'].unique()))
print("cu_eduaction_level -", len(data['cu_eduaction_level'].unique()))
print("big_city -", len(data['big_city'].unique()))

cu_education_level - 5
cu_empl_area - 135
cu_empl_level - 30
cu_eduaction_level - 5
big_city - 4


In [25]:
data_last = data.sort_values('dlk_cob_date').groupby('cif_id').tail(1)
print(data.shape, data_last.shape)
data_last.sort_index().head()
# X.shape

(4161783, 46) (396576, 46)


,cif_id,dlk_cob_date,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,...,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level
10,335992,2018-12-31,10.812928,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,...,1,0,0,0,1192.80000,30.0,150,52.0,1,2
23,198842,2018-12-31,4.287918,SPB,0,NaN,NaN,NaN,1,0,...,1,0,0,0,1801.29600,45.0,23,83.0,0,-100
26,234815,2018-12-31,7.303600,MSK,1,NaN,NaN,SUPPORT STAFF,1,0,...,1,1,1,0,1360.09368,46.0,161,187.0,1,-100
38,232515,2018-12-31,55.700637,OTH,0,03. Высшее,Государственная служба,QUALIFIED STAFF,0,0,...,1,0,0,0,1872.00000,47.0,138,27.0,1,3
53,349326,2018-06-30,30.126211,OTH,1,02. Среднее профессиональное,ЗДРАВООХРАНЕНИЕ,SUPPORT STAFF,0,0,...,1,0,0,0,720.00000,60.0,56,18.0,1,2


# Not updated yet!!!

## Split into validation and train data

In [66]:
X_val = np.append(X[:120], X[-120:], axis=0)
X_train = X[120:-120]
Y_val = np.append(Y[:120], Y[-120:], axis=0)
Y_train = Y[120:-120]
X_val.shape

(240, 20, 65, 1)

## Preparing dataset

In [67]:
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).repeat().shuffle(len(X_val)).batch(32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).repeat().shuffle(len(X_train)).batch(256)

## Preparing model

In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', input_shape=(20, 65, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [69]:
sgd = tf.keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

In [70]:
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])
model.compile(optimizer=sgd,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train model

In [84]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset, steps_per_epoch=2, validation_steps=4)

Train for 2 steps, validate for 4 steps
Epoch 1/50
2/2 [==============================] - 1s 675ms/step - loss: 0.6389 - accuracy: 0.9023 - val_loss: 0.6898 - val_accuracy: 0.5156
Epoch 2/50
2/2 [==============================] - 1s 641ms/step - loss: 0.6393 - accuracy: 0.8848 - val_loss: 0.6766 - val_accuracy: 0.6094
Epoch 3/50
2/2 [==============================] - 1s 620ms/step - loss: 0.6357 - accuracy: 0.9004 - val_loss: 0.6851 - val_accuracy: 0.5234
Epoch 4/50
2/2 [==============================] - 1s 659ms/step - loss: 0.6332 - accuracy: 0.9219 - val_loss: 0.6851 - val_accuracy: 0.5703
Epoch 5/50
2/2 [==============================] - 1s 613ms/step - loss: 0.6331 - accuracy: 0.9102 - val_loss: 0.6830 - val_accuracy: 0.5703
Epoch 6/50
2/2 [==============================] - 1s 617ms/step - loss: 0.6368 - accuracy: 0.9102 - val_loss: 0.6864 - val_accuracy: 0.5234
Epoch 7/50
2/2 [==============================] - 1s 643ms/step - loss: 0.6362 - accuracy: 0.9102 - val_loss: 0.6880 - v